In [1]:
import matplotlib.pyplot as plt

In [2]:
import sys
import re
import pickle
from pathlib import Path
from collections import Counter
from itertools import combinations
from multiprocessing.sharedctypes import RawArray

import torch
import json
import yaml
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from pqdm.processes import pqdm
from IPython.core.display import HTML
from scipy.special import softmax
from scipy.spatial.distance import jensenshannon, cdist
from kneed import KneeLocator

import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as mcolors

sys.path.append("../soup_nuts/models/dvae/")

from dvae import data_iterator, CollapsedMultinomial, DVAE
from utils import load_sparse, compute_to

In [3]:
def load_json(path):
    with open(path) as infile:
        return json.load(infile)

def load_yaml(path):
    with open(path) as infile:
        return yaml.load(infile, Loader=yaml.FullLoader)

def load_text(path):
    with open(path) as infile:
        return [text.strip().split(" ") for text in infile]

def save_json(obj, path):
    with open(path, 'w') as outfile:
        return json.dump(obj, outfile, indent=2)

In [4]:
def topic_est_to_words(topic_words, inv_vocab, n=10):
    return [inv_vocab[idx] for idx in (-topic_words).argsort()[:n]]

### Estimate loading functions

In [5]:
_CUDA_AVAILABLE = torch.cuda.is_available()
_data_cache = {}

def load_mallet_estimates(fpath):
    """
    Load the doc-topic and topic-word estimates from our mallet output folder
    """
    topic_word = np.load(fpath / "beta.npy")
    #topic_word = None 
    # Load the standard mallet document-topic estimate as a numpy matrix
    with open(fpath / "doctopics.txt") as infile:
        doc_topic = np.array([
            [float(x) for x in line.strip().split("\t")[2:]]
            for line in infile
        ])
        
        
    return topic_word, doc_topic, None # TODO: is loss available in mallet?

def load_dvae_estimates(fpath): 
    """
    Loads the dvae model and gets the topic word distribution, then instantiates
    the encoder portion and does a forward pass to get the 
    """
    # get the topic word
    device = torch.device("cuda") if _CUDA_AVAILABLE else torch.device("cpu")

    state_dict = torch.load(fpath / "model.pt", map_location=device)
    beta = state_dict["params"]["decoder$$$eta_layer.weight"]
    topic_word = torch.transpose(beta, 0, 1).cpu().detach().numpy()

    # do a forward pass to get the document topics
    # first instantiate the model and load in the params
    config = load_yaml(fpath / "config.yml")
    
    dvae = DVAE(
        vocab_size=topic_word.shape[1],
        num_topics=config["num_topics"],
        alpha_prior=config["alpha_prior"],
        embeddings_dim=config["encoder_embeddings_dim"],
        hidden_dim=config["encoder_hidden_dim"],
        dropout=config["dropout"],
        cuda=_CUDA_AVAILABLE,
    )
    dvae_dict = {
        k.replace("$$$", "."): v
        for k, v in state_dict['params'].items()
    }
    dvae.load_state_dict(dvae_dict, strict=False)
    dvae.eval()
    turn_off_bn = 1 * (config["epochs_to_anneal_bn"] > 0) # 0 means use BN, > 0 means no BN

    # then load the data for the forward pass
    data_fpath = Path(config["input_dir"], config["train_path"])
    if data_fpath not in _data_cache:
        data = load_sparse(data_fpath).astype(np.float32)
        _data_cache[data_fpath] = data
    else:
        data = _data_cache[data_fpath]
    
    batch_size = config["batch_size"]
    epochs = config["num_epochs"]
    n = data.shape[0]
    train_batches = n // batch_size + 1

    # do the forward pass and collect outputs in an array
    doc_topic = np.zeros((n, config["num_topics"]), dtype=np.float32)
    losses = np.zeros(n, dtype=np.float32)
    for i, x_batch in enumerate(data_iterator(data, batch_size, train_batches)):
        x_batch = x_batch.to(device)
        doc_topic_batch = dvae.encoder(x_batch)
        doc_topic_batch = doc_topic_batch / doc_topic_batch.sum(1, keepdims=True)
        x_recon = dvae.decoder(doc_topic_batch, bn_annealing_factor=turn_off_bn)
        loss_batch = -CollapsedMultinomial(1, probs=x_recon).log_prob(x_batch)

        doc_topic[i * batch_size:(i + 1) * batch_size] = doc_topic_batch.detach().cpu().numpy().astype(np.float32)
        losses[i * batch_size:(i + 1) * batch_size] = loss_batch.detach().cpu().numpy().astype(np.float32)
    return topic_word, doc_topic, losses


def load_etm_estimates(fpath):
    """
    Load the ETM estimates from a model
    """
    pass


def load_estimates(fpath, model_type):
    if model_type == "dvae":
        return load_dvae_estimates(fpath)
    if model_type == "mallet":
        return load_mallet_estimates(fpath)

### collect the runs

In [6]:
# roughly 7 GB RAM for k=100
def get_estimates_over_runs(run_paths, overlap_words, exclude_dups=False):
    doc_topics, topic_words, duplicates = [], [], [] # TODO: change to 3d tensors
    for i, (p, model_type) in enumerate(tqdm(run_paths)):
        t, d, l = load_estimates(p, model_type=model_type)
        
        # located duplicated topics
        sorted_t = np.sort((-t).argsort(axis=1)[:, :overlap_words], axis=1)
        counted_topics = Counter([tuple(t_) for t_ in sorted_t])
        if exclude_dups and max(counted_topics.values()) > 1:
            continue
        doc_topics.append(d)
        topic_words.append(t)
        duplicates.append(sum(c > 1 for c in counted_topics.values()))

    return doc_topics, topic_words, duplicates

#### takes relatively longer to run $\downarrow$

In [7]:
#run_dir = "../runs/outputs/url_partisanship_data"
run_dir = "../runs/outputs/full-mindf_power_law-maxdf_0.9"
dataset = 'wiki' #'url_partisan'

mallet_paths = [
    (p.parent, "mallet")
    for p in Path(run_dir).glob("**/mallet-with-beta/**/doctopics.txt")
    if dataset in str(p)
]
dvae_paths = [
    (p.parent, "dvae") for p in Path(run_dir).glob("**/dvae/**/model.pt")
    if dataset in str(p)
]

# should be independent of the model
config = load_yaml(dvae_paths[0][0] / "config.yml")
data = load_sparse(Path(config["input_dir"], "train.dtm.npz"))
vocab = load_json(Path(config["input_dir"], "vocab.json"))
inv_vocab = {v: k for k, v in vocab.items()}


In [8]:
config['input_dir']

'/workspace/topic-preprocessing/data/wikitext/processed/full-mindf_power_law-maxdf_0.9'

In [96]:
labels_coarse = np.load(Path(config["input_dir"], "train.labels.coarse.npy"))
labels_fine = np.load(Path(config["input_dir"], "train.labels.fine.npy"))
labels_medium = np.load(Path(config["input_dir"], "train.labels.medium.npy"))

In [10]:
# dvae_paths = dvae_paths
# mallet_paths = mallet_paths

In [11]:
len(dvae_paths), len(mallet_paths)

(83, 80)

In [13]:
num_topics = sorted(set(int(re.search("k-([0-9]+)", str(p)).group(1)) for p in dvae_paths))
num_topics

AttributeError: 'NoneType' object has no attribute 'group'

In [21]:
num_topics = [50] 

In [16]:
mallet_paths[0]

(PosixPath('../runs/outputs/full-mindf_power_law-maxdf_0.9/wikitext/k-50/mallet-with-beta/alpha_0.05-beta_0.05-iter_1000-opt_0/42'),
 'mallet')

In [18]:
n_topic_words = 15
overlap_words = 5

#### Gather the betas and thetas for all the mallet and numpy runs $\downarrow$

In [22]:
estimates_dvae, estimates_mallet = {}, {}
for k in num_topics:
    print(f"On k={k}")

    dvae_paths_k = [p for p in dvae_paths if f'k-{k}/' in str(p[0])]
    runs = len(dvae_paths_k)
    if dvae_paths_k:
        doc_topics, topic_words, duplicates = get_estimates_over_runs(dvae_paths_k, overlap_words, exclude_dups=False)
        estimates_dvae[k] = {"doc_topics": doc_topics, "topic_words": topic_words, "duplicates": duplicates}
    
    mallet_paths_k = [p for p in mallet_paths if f'k-{k}/' in str(p[0])]
    if mallet_paths_k:
        doc_topics, topic_words, duplicates = get_estimates_over_runs(mallet_paths_k, overlap_words, exclude_dups=False)
        estimates_mallet[k] = {"doc_topics": doc_topics, "topic_words": topic_words, "duplicates": duplicates}

with open(f"dvae-{dataset}-estimates.pkl", "wb") as outfile:
    pickle.dump(estimates_dvae, outfile)

On k=50


  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

### Find the "user labels" for the documents 

In [23]:
from collections import defaultdict, Counter

In [24]:
doc_topics = estimates_dvae[50]['doc_topics']

In [30]:
labels_fine[:30]

array([454,  -1, 117, 415, 299, 420,  11, 275,  50, 508, 143, 419, 486,
        -1,  -1,  -1, 486, 300,  -1, 324,  99, 112,  -1, 143, 351,  89,
        -1,  18, 337, 390])

In [34]:
len([i for i in labels_medium if i == -1])

6151

In [58]:
user_labels_dvae = {}
for run in range(len(doc_topics)) : 
    user_labels = np.argmax(doc_topics[run], axis=1) 
    user_labels_dvae[run] = user_labels

In [ ]:
gold_labels_coarse = {}
for index, label in enumerate

In [59]:
user_labels_dvae

{0: array([27, 17, 12, ..., 20, 11, 27]),
 1: array([ 1,  8,  6, ..., 35,  3,  4]),
 2: array([15, 43,  3, ..., 40, 26, 15]),
 3: array([15,  9, 43, ..., 40, 39, 15]),
 4: array([15, 36, 25, ..., 40, 26, 24]),
 5: array([45,  4,  7, ...,  1, 13, 45]),
 6: array([38, 41,  1, ...,  7,  0, 38]),
 7: array([38, 32, 37, ..., 43,  0, 38]),
 8: array([49, 28, 46, ...,  5, 41,  9]),
 9: array([47,  8, 23, ...,  9,  6, 29]),
 10: array([49, 41, 46, ...,  5, 13,  9]),
 11: array([49, 28, 46, ...,  5, 13,  9]),
 12: array([15, 43, 23, ..., 40, 26, 24]),
 13: array([17, 25, 42, ...,  7,  0, 38]),
 14: array([19, 44, 46, ...,  3, 13, 19]),
 15: array([27, 47, 12, ..., 48, 11, 27]),
 16: array([49, 39, 14, ...,  3, 30, 49]),
 17: array([49, 17, 12, ...,  3, 30, 49]),
 18: array([49, 43, 46, ...,  5, 13, 49]),
 19: array([26, 21,  9, ...,  7,  2, 26]),
 20: array([23, 40, 29, ...,  0,  6, 23]),
 21: array([49, 25,  1, ...,  3, 30, 49]),
 22: array([24, 36,  3, ..., 40, 26, 24]),
 23: array([37, 36, 1

In [42]:
doc_topics_mallet = estimates_mallet[50]['doc_topics']

In [56]:
len(doc_topics_mallet) 

80

In [60]:
doc_topics_mallet[0].shape

(28472, 50)

In [61]:
user_labels_mallet = {}
for run in range(len(doc_topics_mallet)) : 
    user_labels = np.argmax(doc_topics_mallet[run], axis=1) 
    user_labels_mallet[run] = user_labels

In [62]:
user_labels_mallet

{0: array([46, 43,  0, ..., 36, 20, 23]),
 1: array([11, 33, 28, ..., 14, 27, 11]),
 2: array([34, 26,  5, ..., 30, 10, 34]),
 3: array([24, 18, 28, ..., 19, 46, 24]),
 4: array([37, 15, 48, ...,  5, 35, 37]),
 5: array([11, 26, 17, ..., 13, 10, 11]),
 6: array([44, 39, 46, ..., 29, 25, 44]),
 7: array([ 9, 27, 41, ..., 26, 35,  9]),
 8: array([29, 18,  2, ..., 26, 22, 29]),
 9: array([37, 15, 48, ...,  5, 35, 37]),
 10: array([45,  5,  9, ..., 10, 42, 21]),
 11: array([ 7, 24, 32, ..., 40, 23,  2]),
 12: array([43, 31, 48, ..., 11, 20, 43]),
 13: array([44, 27, 18, ..., 38, 46, 44]),
 14: array([36,  8, 28, ..., 38,  5, 36]),
 15: array([13, 38, 32, ..., 10, 23, 12]),
 16: array([ 8, 28,  1, ..., 39, 32,  8]),
 17: array([43, 31, 48, ..., 11, 20, 43]),
 18: array([34, 42, 43, ..., 30, 24, 34]),
 19: array([41, 15, 38, ..., 42, 24, 41]),
 20: array([22, 41, 11, ..., 40,  5, 22]),
 21: array([ 2, 10,  1, ..., 16, 41,  2]),
 22: array([ 1, 15, 27, ..., 11, 35, 27]),
 23: array([21, 48, 3

In [63]:
np.unique(user_labels_mallet[4], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 560,  304,  974,  290,  477,  615,  665,  416, 1045,  288,  252,
         391,  284,  379, 1644,  454,  987, 1205,  209,  255,  489,  465,
        1024,  543,  841, 1119,  395,  322,  463,  400,  469,  420,  381,
         272,  413, 1152,  362, 1025,  255,  374,  761,  621,  684,  706,
         311,  503,  495,  299,  693,  521]))

In [73]:
bad_labels = list(np.where(labels_coarse == -1)) [0]

In [88]:
min(bad_labels)

1

In [75]:
1 not in bad_labels

False

In [76]:
def purity(user_labels, gold_labels): 
    """
    Calculates the Purity metric as described in https://aclanthology.org/P16-1110/
    "ALTO: Active Learning with Topic Overviews for Speeding Label Induction and Document Labeling"
    
    For sanity check - The purity of any two user labels should be 1
    """
    assert len(user_labels) == len(gold_labels)
    
    user_label_clusters = defaultdict(list)
    for doc_index, label in enumerate(user_labels): 
        if doc_index not in bad_labels: 
            user_label_clusters[label].append(doc_index) 
    
    # main step : 
    purity_sum = 0 
    for cluster_label, user_document_set in user_label_clusters.items(): 
        gold_label_distribution = Counter([gold_labels[i] for i in  user_document_set])
        most_frequent_cluster_freq = sorted(gold_label_distribution.items(), key = lambda x:x[1], reverse=True)[0][1]
        purity_sum += most_frequent_cluster_freq
        
    return purity_sum/len(user_labels) 

In [77]:
from sklearn.metrics import rand_score, normalized_mutual_info_score , adjusted_rand_score

In [78]:
purity(user_labels_dvae[1], gold_labels=labels_coarse) 

0.6565748805844338

In [54]:
adjusted_rand_score(user_labels_dvae[], user_labels_dvae[])

0.42116032385797897

In [202]:
normalized_mutual_info_score(user_labels_dvae[1], user_labels_dvae[0])

0.6056388595988299

In [203]:
metrics = (purity, rand_score, normalized_mutual_info_score)

In [204]:
len(Counter(labels_fine)), len(Counter(labels_coarse))

(2877, 31)

In [205]:
#user_labels_dvae

In [89]:
user_labels_dvae[0]

array([27, 17, 12, ..., 20, 11, 27])

In [83]:
user_labels_dvae[0][~bad_labels]

array([11,  0, 17, ..., 24, 24, 30])

In [86]:
labels_fine

array([ 49, 278, 469, ...,  -1, 497, 387])

In [90]:
mask = np.ones(len(user_labels_dvae[0]), dtype=bool) # all elements included/True.
mask[bad_labels] = False 

In [91]:
user_labels_dvae[0]

array([27, 17, 12, ..., 20, 11, 27])

In [92]:
user_labels_dvae[0][mask]

array([27, 12, 42, ..., 20, 11, 27])

In [95]:
len(labels_coarse)

6151

In [97]:
labels_fine = labels_fine[mask]
labels_coarse = labels_coarse[mask]
labels_medium = labels_medium[mask]

In [98]:
def get_dataframe(user_labels_list, model) :
    stuff = []
    for index in range(len(user_labels_list)) : 
        user_labels = user_labels_list[index][mask]

        d = {
        "model": f"{model}",
        "index": f"{model}_{index}",
        "rand_fine" : adjusted_rand_score(user_labels, labels_fine), 
        "rand_coarse" : adjusted_rand_score(user_labels, labels_coarse), 
        "nmi_fine": normalized_mutual_info_score(user_labels, labels_fine),
        "nmi_coarse": normalized_mutual_info_score(user_labels, labels_coarse),
        "purity_fine": purity(user_labels, labels_fine),
        "purity_coarse": purity(user_labels, labels_coarse)
        }

    #     print(f"Model {index}\n\t Rand Scores: {rand_coarse:.3f},{rand_fine:.3f} \
    #              \tNMI Scores: {nmi_coarse:.3f},{nmi_fine:.3f} \
    #              \tPurity Scores: {purity_coarse:.3f},{purity_fine:.3f}")

        stuff.append(d)
        
    df = pd.DataFrame(stuff)

    return df

In [99]:
dvae = get_dataframe(user_labels_dvae, "dvae")

In [100]:
dvae.head()

,model,index,rand_fine,rand_coarse,nmi_fine,nmi_coarse,purity_fine,purity_coarse
0,dvae,dvae_0,0.126609,0.488711,0.581835,0.655661,0.138166,0.620985
1,dvae,dvae_1,0.232784,0.304770,0.665838,0.618177,0.222660,0.658259
2,dvae,dvae_2,0.234363,0.314916,0.660457,0.617792,0.214462,0.653152
3,dvae,dvae_3,0.240250,0.302758,0.670089,0.624500,0.223646,0.662605
4,dvae,dvae_4,0.220073,0.288681,0.640260,0.594534,0.206398,0.642624


In [101]:
mallet = get_dataframe(user_labels_mallet, "mallet")

In [102]:
mallet.head()

,model,index,rand_fine,rand_coarse,nmi_fine,nmi_coarse,purity_fine,purity_coarse
0,mallet,mallet_0,0.222347,0.305325,0.640946,0.601537,0.215582,0.646835
1,mallet,mallet_1,0.242516,0.311352,0.660462,0.611603,0.216612,0.650688
2,mallet,mallet_2,0.229226,0.310525,0.658138,0.613428,0.220241,0.656512
3,mallet,mallet_3,0.225675,0.341299,0.657734,0.614826,0.214910,0.651405
4,mallet,mallet_4,0.229480,0.329078,0.659023,0.610697,0.213879,0.653555


In [103]:
df = pd.concat([dvae, mallet], ignore_index=True)

In [104]:
df.shape

(162, 8)

In [105]:
df.head()

,model,index,rand_fine,rand_coarse,nmi_fine,nmi_coarse,purity_fine,purity_coarse
0,dvae,dvae_0,0.126609,0.488711,0.581835,0.655661,0.138166,0.620985
1,dvae,dvae_1,0.232784,0.304770,0.665838,0.618177,0.222660,0.658259
2,dvae,dvae_2,0.234363,0.314916,0.660457,0.617792,0.214462,0.653152
3,dvae,dvae_3,0.240250,0.302758,0.670089,0.624500,0.223646,0.662605
4,dvae,dvae_4,0.220073,0.288681,0.640260,0.594534,0.206398,0.642624


In [107]:
df.to_csv("coverage_results_for_viz_wiki.csv", index=False) 

In [106]:
!pwd

/workspace/topic-preprocessing/analysis


In [157]:
dvae_stuff = []
for index in range(len(user_labels_dvae)) : 
    user_labels = user_labels_dvae[index]
    
    d = {
    "rand_fine" : adjusted_rand_score(user_labels, labels_fine), 
    "rand_coarse" : adjusted_rand_score(user_labels, labels_coarse), 
    "nmi_fine": normalized_mutual_info_score(user_labels, labels_fine),
    "nmi_coarse": normalized_mutual_info_score(user_labels, labels_coarse),
    "purity_fine": purity(user_labels, labels_fine),
    "purity_coarse" = purity(user_labels, labels_coarse)
    "model":"dvae",
    "index":f"dvae_{index}"
    }
    
#     print(f"Model {index}\n\t Rand Scores: {rand_coarse:.3f},{rand_fine:.3f} \
#              \tNMI Scores: {nmi_coarse:.3f},{nmi_fine:.3f} \
#              \tPurity Scores: {purity_coarse:.3f},{purity_fine:.3f}")

    dvae_stuff.append(d)

Model 0
	 Rand Scores: 0.156,0.317              	NMI Scores: 0.365,0.447              	Purity Scores: 0.516,0.322
Model 1
	 Rand Scores: 0.174,0.395              	NMI Scores: 0.376,0.468              	Purity Scores: 0.536,0.334
Model 2
	 Rand Scores: 0.143,0.257              	NMI Scores: 0.387,0.466              	Purity Scores: 0.550,0.353
Model 3
	 Rand Scores: 0.179,0.351              	NMI Scores: 0.384,0.485              	Purity Scores: 0.545,0.355
Model 4
	 Rand Scores: 0.144,0.282              	NMI Scores: 0.322,0.393              	Purity Scores: 0.508,0.312


In [231]:
for index in range(len(user_labels_dvae)) : 
    user_labels = user_labels_dvae[index]
    rand_fine = adjusted_rand_score(user_labels, labels_fine)
    rand_coarse = adjusted_rand_score(user_labels, labels_coarse)
    nmi_fine = normalized_mutual_info_score(user_labels, labels_fine)
    nmi_coarse = normalized_mutual_info_score(user_labels, labels_coarse)
    purity_fine = purity(user_labels, labels_fine)
    purity_coarse = purity(user_labels, labels_coarse)
    
    print(f"Model {index}\n\t Rand Scores: {rand_coarse:.3f},{rand_fine:.3f} \
             \tNMI Scores: {nmi_coarse:.3f},{nmi_fine:.3f} \
             \tPurity Scores: {purity_coarse:.3f},{purity_fine:.3f}")

Model 0
	 Rand Scores: 0.156,0.317              	NMI Scores: 0.365,0.447              	Purity Scores: 0.516,0.322
Model 1
	 Rand Scores: 0.174,0.395              	NMI Scores: 0.376,0.468              	Purity Scores: 0.536,0.334
Model 2
	 Rand Scores: 0.143,0.257              	NMI Scores: 0.387,0.466              	Purity Scores: 0.550,0.353
Model 3
	 Rand Scores: 0.179,0.351              	NMI Scores: 0.384,0.485              	Purity Scores: 0.545,0.355
Model 4
	 Rand Scores: 0.144,0.282              	NMI Scores: 0.322,0.393              	Purity Scores: 0.508,0.312
Model 5
	 Rand Scores: 0.203,0.404              	NMI Scores: 0.380,0.481              	Purity Scores: 0.546,0.361
Model 6
	 Rand Scores: 0.156,0.336              	NMI Scores: 0.367,0.444              	Purity Scores: 0.534,0.329
Model 7
	 Rand Scores: 0.148,0.252              	NMI Scores: 0.364,0.436              	Purity Scores: 0.521,0.308
Model 8
	 Rand Scores: 0.164,0.316              	NMI Scores: 0.392,0.477              	P

Model 72
	 Rand Scores: 0.142,0.292              	NMI Scores: 0.373,0.478              	Purity Scores: 0.539,0.352
Model 73
	 Rand Scores: 0.141,0.253              	NMI Scores: 0.383,0.458              	Purity Scores: 0.540,0.339
Model 74
	 Rand Scores: 0.194,0.380              	NMI Scores: 0.393,0.474              	Purity Scores: 0.552,0.336
Model 75
	 Rand Scores: 0.160,0.310              	NMI Scores: 0.386,0.474              	Purity Scores: 0.555,0.350
Model 76
	 Rand Scores: 0.166,0.337              	NMI Scores: 0.381,0.476              	Purity Scores: 0.547,0.348
Model 77
	 Rand Scores: 0.162,0.323              	NMI Scores: 0.388,0.484              	Purity Scores: 0.554,0.357
Model 78
	 Rand Scores: 0.171,0.355              	NMI Scores: 0.391,0.493              	Purity Scores: 0.557,0.365
Model 79
	 Rand Scores: 0.136,0.241              	NMI Scores: 0.346,0.414              	Purity Scores: 0.515,0.294
Model 80
	 Rand Scores: 0.151,0.292              	NMI Scores: 0.358,0.443       

In [230]:
for index in range(len(user_labels_mallet)) : 
    user_labels = user_labels_mallet[index]
    rand_fine = adjusted_rand_score(user_labels, labels_fine)
    rand_coarse = adjusted_rand_score(user_labels, labels_coarse)
    nmi_fine = normalized_mutual_info_score(user_labels, labels_fine)
    nmi_coarse = normalized_mutual_info_score(user_labels, labels_coarse)
    purity_fine = purity(user_labels, labels_fine)
    purity_coarse = purity(user_labels, labels_coarse)
    
    print(f"Model {index}\n\t Rand Scores: {rand_coarse:.3f},{rand_fine:.3f} \
             \tNMI Scores: {nmi_coarse:.3f},{nmi_fine:.3f} \
             \tPurity Scores: {purity_coarse:.3f},{purity_fine:.3f}")

Model 0
	 Rand Scores: 0.183,0.350              	NMI Scores: 0.371,0.456              	Purity Scores: 0.546,0.327
Model 1
	 Rand Scores: 0.178,0.343              	NMI Scores: 0.371,0.460              	Purity Scores: 0.543,0.333
Model 2
	 Rand Scores: 0.154,0.318              	NMI Scores: 0.374,0.456              	Purity Scores: 0.552,0.332
Model 3
	 Rand Scores: 0.190,0.408              	NMI Scores: 0.378,0.468              	Purity Scores: 0.552,0.343
Model 4
	 Rand Scores: 0.194,0.423              	NMI Scores: 0.370,0.468              	Purity Scores: 0.541,0.349
Model 5
	 Rand Scores: 0.167,0.352              	NMI Scores: 0.371,0.467              	Purity Scores: 0.552,0.353
Model 6
	 Rand Scores: 0.190,0.406              	NMI Scores: 0.363,0.445              	Purity Scores: 0.531,0.320
Model 7
	 Rand Scores: 0.199,0.430              	NMI Scores: 0.372,0.466              	Purity Scores: 0.548,0.341
Model 8
	 Rand Scores: 0.183,0.359              	NMI Scores: 0.369,0.458              	P

Model 72
	 Rand Scores: 0.193,0.408              	NMI Scores: 0.372,0.457              	Purity Scores: 0.544,0.329
Model 73
	 Rand Scores: 0.168,0.354              	NMI Scores: 0.372,0.468              	Purity Scores: 0.553,0.353
Model 74
	 Rand Scores: 0.173,0.345              	NMI Scores: 0.380,0.470              	Purity Scores: 0.555,0.354
Model 75
	 Rand Scores: 0.201,0.422              	NMI Scores: 0.368,0.459              	Purity Scores: 0.539,0.333
Model 76
	 Rand Scores: 0.195,0.427              	NMI Scores: 0.371,0.467              	Purity Scores: 0.543,0.345
Model 77
	 Rand Scores: 0.170,0.340              	NMI Scores: 0.370,0.457              	Purity Scores: 0.545,0.329
Model 78
	 Rand Scores: 0.197,0.433              	NMI Scores: 0.380,0.472              	Purity Scores: 0.552,0.359
Model 79
	 Rand Scores: 0.192,0.417              	NMI Scores: 0.379,0.470              	Purity Scores: 0.554,0.344
Model 80
	 Rand Scores: 0.197,0.431              	NMI Scores: 0.370,0.465       

In [ ]:
users